In [9]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import csv
from datetime import datetime, timedelta
import time
import random

def get_random_proxy():
    # Add your proxy list here
    proxies = [
        'http://proxy1.example.com:8080',
        'http://proxy2.example.com:8080',
        # Add more proxies as needed
    ]
    return {'http': random.choice(proxies), 'https': random.choice(proxies)} if proxies else None

def google_news_search(keyword, date, max_retries=5, base_delay=60):
    formatted_date = date.strftime("%Y/%m/%d")
    encoded_keyword = urllib.parse.quote(keyword)
    url = (
        f"https://www.google.com/search?q={encoded_keyword}&tbm=nws"
        f"&tbs=cdr:1,cd_min:{formatted_date},cd_max:{formatted_date}"
    )
    headers = {
        "User-Agent": random.choice([
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        ]),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "Referer": "https://www.google.com/",
        "DNT": "1"
    }

    for attempt in range(max_retries):
        try:
            if attempt > 0:
                # Exponential backoff with jitter
                delay = base_delay * (2 ** attempt) + random.uniform(10, 30)
                print(f"第 {attempt + 1} 次重試，等待 {delay:.1f} 秒...")
                time.sleep(delay)

            proxies = get_random_proxy()
            response = requests.get(url, headers=headers, proxies=proxies, timeout=30)
            print(f"搜尋網址: {url}")
            print(f"回應狀態: {response.status_code}")
            print(f"回應內容大小: {len(response.text)}")

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                results = []

                # 方法 1：新格式 (常見於舊新聞)
                for item in soup.select('div.SoaBEf'):
                    title_tag = item.select_one('div.MBeuO')
                    link_tag = item.find('a', href=True)
                    if title_tag and link_tag:
                        results.append({
                            'date': formatted_date,
                            'keyword': keyword,
                            'title': title_tag.text.strip(),
                            'link': link_tag['href']
                        })

                # 方法 2：舊格式 (dbsr)
                for item in soup.select('div.dbsr'):
                    title_tag = item.select_one('div.JheGif span') or item.select_one('div.JheGif')
                    link = item.a['href']
                    title = title_tag.text if title_tag else "No title"
                    results.append({
                        'date': formatted_date,
                        'keyword': keyword,
                        'title': title,
                        'link': link
                    })

                return results
            elif response.status_code == 429:
                print(f"遇到速率限制 (429 錯誤)，進行第 {attempt + 1} 次重試")
                # Add additional delay for rate limit
                time.sleep(random.uniform(120, 180))
                continue
            else:
                print(f"遇到未預期的狀態碼: {response.status_code}")

        except Exception as e:
            print(f"錯誤 (第 {attempt + 1} 次): {str(e)}")
            time.sleep(random.uniform(30, 60))

    print("已達到最大重試次數，跳過此日期")
    return []

def save_results_to_csv(results, filename='google_news_results.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=['date', 'keyword', 'title', 'link'])
        writer.writeheader()
        for row in results:
            writer.writerow(row)

if __name__ == "__main__":
    keyword = "鴻海"
    start_date = datetime.strptime("2010-10-19", "%Y-%m-%d")
    end_date = datetime.strptime("2010-10-22", "%Y-%m-%d")

    all_results = []
    current_date = start_date

    while current_date <= end_date:
        print(f"\n搜尋 {keyword} 於 {current_date.strftime('%Y-%m-%d')}...")
        results = google_news_search(keyword, current_date)
        print(f"找到 {len(results)} 則新聞")
        all_results.extend(results)

        # 增加間隔時間至 90-150 秒
        if current_date < end_date:
            sleep_time = 90 + random.random() * 60
            print(f"等待 {sleep_time:.1f} 秒後進行下一次搜尋...")
            time.sleep(sleep_time)

        current_date += timedelta(days=1)

    save_results_to_csv(all_results)
    print(f"\n✅ 完成，共儲存 {len(all_results)} 則新聞至 CSV 檔。")



搜尋 鴻海 於 2010-10-19...
錯誤 (第 1 次): HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=%E9%B4%BB%E6%B5%B7&tbm=nws&tbs=cdr:1,cd_min:2010/10/19,cd_max:2010/10/19 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb969797250>: Failed to establish a new connection: [Errno -2] Name or service not known')))
錯誤 (第 1 次): HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=%E9%B4%BB%E6%B5%B7&tbm=nws&tbs=cdr:1,cd_min:2010/10/19,cd_max:2010/10/19 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb969797250>: Failed to establish a new connection: [Errno -2] Name or service not known')))
第 2 次重試，等待 134.2 秒...
第 2 次重試，等待 134.2 秒...
錯誤 (第 2 次): HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=%E9%B4%BB%E6%B5%B7&tbm=nws&tbs=cdr:1,cd_mi

KeyboardInterrupt: 

# playwright version

In [56]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from datetime import datetime, timedelta
from playwright.async_api import async_playwright
import csv
import random
import time

MAX_NEWS = 1000  # 最多抓 1000 筆

def human_delay(a=1.5, b=3.0):
    time.sleep(random.uniform(a, b))


async def google_news_search_all_pages(keyword, date1, date2):
    formatted_date1 = date1.strftime("%m/%d/%Y")
    formatted_date2 = date2.strftime("%m/%d/%Y")

    url = (
        f"https://www.google.com.tw/search?q={keyword}&hl=zh-TW&gl=tw&tbm=nws"
        f"&tbs=cdr:1,cd_min:{formatted_date1},cd_max:{formatted_date2}"
    )

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto(url)

        await page.wait_for_timeout(3000)
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await page.wait_for_timeout(2000)

        results = []
        page_num = 1

        while True:
            print(f"📄 擷取第 {page_num} 頁 ({date1.date()} ~ {date2.date()})")

            articles = await page.query_selector_all('div.SoaBEf, div.dbsr')
            for article in articles:
                try:
                    title_elem = await article.query_selector('div.MBeuO, div.JheGif')
                    link_elem = await article.query_selector('a')
                    date_elem = await article.query_selector('div[class*="OSrXXb"] span')

                    title = await title_elem.inner_text() if title_elem else ''
                    link = await link_elem.get_attribute('href') if link_elem else ''
                    pub_date = await date_elem.inner_text() if date_elem else 'N/A'

                    results.append({
                        'date': f"{formatted_date1}~{formatted_date2}",
                        'keyword': keyword,
                        'title': title,
                        'link': link,
                        'publish_date': pub_date
                    })

                    if len(results) >= MAX_NEWS:
                        break

                except Exception:
                    continue

            if len(results) >= MAX_NEWS:
                break

            try:
                next_button = await page.query_selector('#pnnext')
                if next_button:
                    await next_button.click()
                    page_num += 1
                    await page.wait_for_timeout(1500)
                else:
                    break
            except Exception:
                break

        await browser.close()
        return results


def save_results_to_csv(results, filename='google_news_results_playwright.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=['date', 'keyword', 'title', 'link', 'publish_date'])
        writer.writeheader()
        for row in results:
            writer.writerow(row)


async def main():
    keyword = "鴻海財經"
    start_date = datetime.strptime("2010-10-19", "%Y-%m-%d")
    end_date = datetime.strptime("2024-10-22", "%Y-%m-%d")
    delta = timedelta(days=30)

    print(f"\n🔍 搜尋 {keyword} - {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")

    all_results = []
    current_date = start_date

    while current_date < end_date and len(all_results) < MAX_NEWS:
        next_date = min(current_date + delta, end_date)
        results = await google_news_search_all_pages(keyword, current_date, next_date)
        all_results.extend(results)

        print(f"🧮 累積 {len(all_results)} 筆資料")

        if len(all_results) >= MAX_NEWS:
            break

        current_date = next_date + timedelta(days=1)

    save_results_to_csv(all_results[:MAX_NEWS])
    print(f"\n✅ 完成，共儲存 {len(all_results[:MAX_NEWS])} 則新聞到 CSV 檔。")


# ▶️ 執行
await main()



🔍 搜尋 鴻海財經 - 2010-10-19 ~ 2024-10-22
📄 擷取第 1 頁 (2010-10-19 ~ 2010-11-18)
🧮 累積 0 筆資料
📄 擷取第 1 頁 (2010-11-19 ~ 2010-12-19)
🧮 累積 0 筆資料
📄 擷取第 1 頁 (2010-12-20 ~ 2011-01-19)
🧮 累積 1 筆資料
📄 擷取第 1 頁 (2011-01-20 ~ 2011-02-19)
🧮 累積 1 筆資料
📄 擷取第 1 頁 (2011-02-20 ~ 2011-03-22)
🧮 累積 1 筆資料
📄 擷取第 1 頁 (2011-03-23 ~ 2011-04-22)
🧮 累積 1 筆資料
📄 擷取第 1 頁 (2011-04-23 ~ 2011-05-23)
🧮 累積 3 筆資料
📄 擷取第 1 頁 (2011-05-24 ~ 2011-06-23)
🧮 累積 6 筆資料
📄 擷取第 1 頁 (2011-06-24 ~ 2011-07-24)
🧮 累積 8 筆資料
📄 擷取第 1 頁 (2011-07-25 ~ 2011-08-24)
🧮 累積 9 筆資料
📄 擷取第 1 頁 (2011-08-25 ~ 2011-09-24)
🧮 累積 10 筆資料
📄 擷取第 1 頁 (2011-09-25 ~ 2011-10-25)
🧮 累積 12 筆資料
📄 擷取第 1 頁 (2011-10-26 ~ 2011-11-25)
🧮 累積 12 筆資料
📄 擷取第 1 頁 (2011-11-26 ~ 2011-12-26)
🧮 累積 16 筆資料
📄 擷取第 1 頁 (2011-12-27 ~ 2012-01-26)
🧮 累積 17 筆資料
📄 擷取第 1 頁 (2012-01-27 ~ 2012-02-26)
🧮 累積 18 筆資料
📄 擷取第 1 頁 (2012-02-27 ~ 2012-03-28)
🧮 累積 22 筆資料
📄 擷取第 1 頁 (2012-03-29 ~ 2012-04-28)
🧮 累積 25 筆資料
📄 擷取第 1 頁 (2012-04-29 ~ 2012-05-29)
🧮 累積 30 筆資料
📄 擷取第 1 頁 (2012-05-30 ~ 2012-06-29)
🧮 累積 31 筆資料
📄 擷取第 1 頁 (20

Future exception was never retrieved
future: <Future finished exception=Error('Connection closed')>
playwright._impl._api_types.Error: Connection closed


📄 擷取第 2 頁 (2017-08-03 ~ 2017-09-02)
🧮 累積 581 筆資料
📄 擷取第 1 頁 (2017-09-03 ~ 2017-10-03)
📄 擷取第 2 頁 (2017-09-03 ~ 2017-10-03)
📄 擷取第 3 頁 (2017-09-03 ~ 2017-10-03)
🧮 累積 600 筆資料
📄 擷取第 1 頁 (2017-10-04 ~ 2017-11-03)
📄 擷取第 2 頁 (2017-10-04 ~ 2017-11-03)
📄 擷取第 3 頁 (2017-10-04 ~ 2017-11-03)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2017-11-04 ~ 2017-12-04)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2017-12-05 ~ 2018-01-04)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-01-05 ~ 2018-02-04)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-02-05 ~ 2018-03-07)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-03-08 ~ 2018-04-07)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-04-08 ~ 2018-05-08)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-05-09 ~ 2018-06-08)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-06-09 ~ 2018-07-09)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-07-10 ~ 2018-08-09)
🧮 累積 621 筆資料
📄 擷取第 1 頁 (2018-08-10 ~ 2018-09-09)


CancelledError: 

使用不同年份分開

In [26]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import csv
import random
import time
from datetime import datetime, timedelta
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

# 🔧 可調參數
MAX_NEWS = 1000
DELAY_RANGE = (1.5, 3.0)
HEADLESS = False
CSV_FILE = '聯發科爬到的新聞資料還有內文/google_news_results_playwright 2024-01-01_2024-12-31.csv'



def human_delay(a=1.5, b=3.0):
    time.sleep(random.uniform(a, b))


async def google_news_search_all_pages(keyword, date1, date2):
    formatted_date1 = date1.strftime("%m/%d/%Y")
    formatted_date2 = date2.strftime("%m/%d/%Y")

    url = (
        f"https://www.google.com.tw/search?q={keyword}&hl=zh-TW&gl=tw&tbm=nws"
        f"&tbs=cdr:1,cd_min:{formatted_date1},cd_max:{formatted_date2}"
    )

    results = []
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=HEADLESS)
            context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                                           "AppleWebKit/537.36 (KHTML, like Gecko) "
                                                           "Chrome/122.0.0.0 Safari/537.36")
            page = await context.new_page()

            await page.goto(url)
            await page.wait_for_timeout(3000)

            await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
            await page.wait_for_timeout(2000)

            page_num = 1
            while True:
                print(f"📄 擷取第 {page_num} 頁 ({date1.date()} ~ {date2.date()})")

                articles = await page.query_selector_all('div.SoaBEf, div.dbsr')
                for article in articles:
                    try:
                        title_elem = await article.query_selector('div.MBeuO, div.JheGif')
                        link_elem = await article.query_selector('a')
                        date_elem = await article.query_selector('div[class*="OSrXXb"] span')

                        title = await title_elem.inner_text() if title_elem else ''
                        link = await link_elem.get_attribute('href') if link_elem else ''
                        pub_date = await date_elem.inner_text() if date_elem else 'N/A'

                        results.append({
                            'date': f"{formatted_date1}~{formatted_date2}",
                            'keyword': keyword,
                            'title': title,
                            'link': link,
                            'publish_date': pub_date
                        })

                        if len(results) >= MAX_NEWS:
                            break

                    except Exception as e:
                        print(f"⚠️ 擷取新聞時錯誤：{e}")
                        continue

                if len(results) >= MAX_NEWS:
                    break

                try:
                    next_button = await page.query_selector('#pnnext')
                    if next_button:
                        await next_button.click()
                        page_num += 1
                        await page.wait_for_timeout(1500)
                    else:
                        break
                except PlaywrightTimeoutError:
                    print("⚠️ 找不到下一頁或逾時")
                    break

            await browser.close()

    except Exception as e:
        print(f"🚨 瀏覽器錯誤：{e}")

    return results


def save_results_to_csv(results, filename=CSV_FILE):
    try:
        with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=['date', 'keyword', 'title', 'link', 'publish_date'])
            writer.writeheader()
            for row in results:
                writer.writerow(row)
    except Exception as e:
        print(f"❌ 寫入 CSV 錯誤：{e}")


async def main():
    keyword = "聯發科 財經"
    start_date = datetime.strptime("2024-01-01", "%Y-%m-%d")
    end_date = datetime.strptime("2024-12-31", "%Y-%m-%d")
    delta = timedelta(days=30)

    print(f"\n🔍 搜尋「{keyword}」 - {start_date.date()} 到 {end_date.date()}")

    all_results = []
    current_date = start_date

    while current_date < end_date and len(all_results) < MAX_NEWS:
        next_date = min(current_date + delta, end_date)

        try:
            results = await google_news_search_all_pages(keyword, current_date, next_date)
            all_results.extend(results)
            print(f"🧮 累積 {len(all_results)} 筆資料")

            # ✅ 每次取得新資料就即時寫入 CSV
            save_results_to_csv(all_results[:MAX_NEWS])

        except Exception as e:
            print(f"❗ 處理 {current_date.date()} ~ {next_date.date()} 錯誤：{e}")

        if len(all_results) >= MAX_NEWS:
            break

        current_date = next_date + timedelta(days=1)
        human_delay(*DELAY_RANGE)

    print(f"\n✅ 完成，共儲存 {len(all_results[:MAX_NEWS])} 則新聞到 CSV 檔。")


# ▶️ 執行
await main()



🔍 搜尋「聯發科 財經」 - 2024-01-01 到 2024-12-31
📄 擷取第 1 頁 (2024-01-01 ~ 2024-01-31)
📄 擷取第 2 頁 (2024-01-01 ~ 2024-01-31)
🧮 累積 10 筆資料
📄 擷取第 1 頁 (2024-02-01 ~ 2024-03-02)
📄 擷取第 2 頁 (2024-02-01 ~ 2024-03-02)
🧮 累積 20 筆資料
📄 擷取第 1 頁 (2024-03-03 ~ 2024-04-02)
📄 擷取第 2 頁 (2024-03-03 ~ 2024-04-02)
🧮 累積 30 筆資料
📄 擷取第 1 頁 (2024-04-03 ~ 2024-05-03)
📄 擷取第 2 頁 (2024-04-03 ~ 2024-05-03)
🧮 累積 40 筆資料
📄 擷取第 1 頁 (2024-05-04 ~ 2024-06-03)
📄 擷取第 2 頁 (2024-05-04 ~ 2024-06-03)
🧮 累積 50 筆資料
📄 擷取第 1 頁 (2024-06-04 ~ 2024-07-04)
📄 擷取第 2 頁 (2024-06-04 ~ 2024-07-04)
🧮 累積 60 筆資料
📄 擷取第 1 頁 (2024-07-05 ~ 2024-08-04)
📄 擷取第 2 頁 (2024-07-05 ~ 2024-08-04)
🧮 累積 70 筆資料
📄 擷取第 1 頁 (2024-08-05 ~ 2024-09-04)
📄 擷取第 2 頁 (2024-08-05 ~ 2024-09-04)
🧮 累積 80 筆資料
📄 擷取第 1 頁 (2024-09-05 ~ 2024-10-05)
📄 擷取第 2 頁 (2024-09-05 ~ 2024-10-05)
🧮 累積 90 筆資料
📄 擷取第 1 頁 (2024-10-06 ~ 2024-11-05)
📄 擷取第 2 頁 (2024-10-06 ~ 2024-11-05)
🧮 累積 100 筆資料
📄 擷取第 1 頁 (2024-11-06 ~ 2024-12-06)
📄 擷取第 2 頁 (2024-11-06 ~ 2024-12-06)
🧮 累積 110 筆資料
📄 擷取第 1 頁 (2024-12-07 ~ 2024-12-31

把所有網站連結合併成一個csv

In [27]:
import pandas as pd
import glob

# 找出所有目標 CSV 檔案
file_paths = sorted(glob.glob('聯發科爬到的新聞資料還有內文/google_news_results_playwright*.csv'))

# 合併所有檔案
all_data = []
for file in file_paths:
    df = pd.read_csv(file)
    all_data.append(df)

# 合併成一個 DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# 將日期欄位轉成 datetime 格式並排序（假設欄位叫 'date'）
merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')
merged_df = merged_df.sort_values(by='date')

# 儲存為新檔案
merged_df.to_csv('聯發科爬到的新聞資料還有內文/google_news_results_playwright_merged_sorted.csv', index=False)


/tmp/ipykernel_1621499/2683665285.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')


In [2]:
import asyncio
from playwright.async_api import async_playwright

async def fetch_articles():
    test_urls = {
        "wealth": "https://www.wealth.com.tw/articles/de60abf4-f462-4fca-9976-5d2c7c7185da",
        "chinatimes": "https://www.chinatimes.com/newspapers/20110428000002-260202",
        "ltn": "https://ec.ltn.com.tw/article/paper/499221"
    }

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        for site, url in test_urls.items():
            try:
                await page.goto(url, timeout=20000)
                await page.wait_for_load_state('domcontentloaded')

                content_element = None

                if "wealth.com.tw" in url:
                    await page.wait_for_selector(".qdr-paragraph", timeout=10000)
                    paragraph_elements = await page.query_selector_all(".qdr-paragraph span[style*='white-space: pre-wrap']")
                    content_list = [await elem.inner_text() for elem in paragraph_elements]
                    content = "\n".join(content_list)

                elif "chinatimes.com" in url:
                    try:
                        await page.wait_for_selector(".article-body", timeout=8000)
                        content_element = await page.query_selector(".article-body")
                    except:
                        try:
                            await page.wait_for_selector(".article-content", timeout=8000)
                            content_element = await page.query_selector(".article-content")
                        except:
                            content_element = None

                elif "ltn.com.tw" in url:
                    await page.wait_for_selector(".text", timeout=10000)
                    content_element = await page.query_selector(".text")

                if content_element:
                    content = await content_element.inner_text()
                    print(f"\n✅ {site} 內文前500字：\n", content.strip()[:500])
                else:
                    print(f"❌ {site} 找不到內容元素")

            except Exception as e:
                print(f"❌ {site} 發生錯誤：{e}")

        await browser.close()

await fetch_articles()


Error: 
╔════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Looks like you launched a headed browser without having a XServer running.                     ║
║ Set either 'headless: true' or use 'xvfb-run <your-playwright-app>' before running Playwright. ║
║                                                                                                ║
║ <3 Playwright Team                                                                             ║
╚════════════════════════════════════════════════════════════════════════════════════════════════╝
=========================== logs ===========================
<launching> /home/r11011101/.cache/ms-playwright/chromium-1080/chrome-linux/chrome --disable-field-trial-config --disable-background-networking --enable-features=NetworkService,NetworkServiceInProcess --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable

In [ ]:
import requests, trafilatura
from newspaper import Article
from readability import Document
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import asyncio

test_urls = {
    "wealth": "https://www.wealth.com.tw/articles/de60abf4-f462-4fca-9976-5d2c7c7185da",
    "chinatimes": "https://www.chinatimes.com/newspapers/20110428000002-260202",
    "ltn": "https://ec.ltn.com.tw/article/paper/499221"
}

async def fallback_with_browser(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto(url, timeout=30000)

        # 滾動頁面讓內容完全載入
        for i in range(10):  # 可根據實際需求調整次數
            await page.mouse.wheel(0, 1000)
            await page.wait_for_timeout(500)

        # 等待所有懶加載內容載入
        await page.wait_for_timeout(3000)

        # 擷取全文
        text = await page.inner_text("body")
        await browser.close()
        print("✅ Playwright 擷取成功：\n", text[:1500])


async def extract_from_url(name, url):
    print(f"\n🚀 正在處理：{name} - {url}")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/114.0.0.0 Safari/537.36"}

    try:
        html = requests.get(url, headers=headers, timeout=15).text
    except:
        html = None

    # 方法 1：trafilatura
    if html:
        text = trafilatura.extract(html)
        if text and len(text.strip()) > 100:
            print("✅ Trafilatura 擷取成功：\n", text[:1500])
            return

    # 方法 2：newspaper3k
    try:
        article = Article(url); article.download(); article.parse()
        if article.text.strip():
            print("✅ Newspaper 擷取成功：\n", article.text[:1500])
            return
    except:
        pass

    # 方法 3：readability
    try:
        summary = BeautifulSoup(Document(html).summary(), "html.parser").get_text()
        if len(summary.strip()) > 100:
            print("✅ Readability 擷取成功：\n", summary[:1500])
            return
    except:
        pass

    # 方法 4：瀏覽器備援
    print("🚨 所有方法失敗，改用瀏覽器擷取...")
    await fallback_with_browser(url)

async def main():
    for name, url in test_urls.items():
        await extract_from_url(name, url)

# ✅ Jupyter Notebook 中執行這一行：
await main()



🚀 正在處理：wealth - https://www.wealth.com.tw/articles/de60abf4-f462-4fca-9976-5d2c7c7185da
🚨 所有方法失敗，改用瀏覽器擷取...
✅ Playwright 擷取成功：
 財訊商城
講座｜課程
訂閱電子報
會員中心
登出
財經
政經
金融圈
科技
地產
生醫
國際
企業
財經指標
投資
財經茶水間
投資理財
基金報報
藝術投資
生活
健康醫療
美食旅遊
品味人生
老謝財經茶水間
都更全都通
New
獨家專訪賴清德
訂閱雜誌
全站搜尋
為提供您更多優質的內容，本網站使用 cookies 分析技術。若繼續閱覽本網站內容，即表示您同意我們使用 cookies，關於更多 cookies 以及相關政策更新資訊請閱讀我們的 隱私權政策 。
我知道了
HOT
個資被看光?
無人機進軍國際
財訊週年慶
投資理財
知名部落客綠角 買ETF跟著指數賺 不用設停損
2011/01/12
08:00
文｜
鄧麗萍
知名部落客綠角 買ETF跟著指數賺 不用設停損
分享
分享
分享
複製連結

知名部落客綠角透過ETF投資台股，不擔心挑到壞股票、爛基金，金融海嘯前買進0050，雖然大跌4成，但愈買愈低，如今總報酬率超過30%。在ETF操作上，綠角嚴守資產配置，跌深加碼、漲多賣出，跟著指數穩穩賺。

畢業於台大醫學系、現任眼科主治醫師的知名財經部落客綠角，對於指數化投資（ETF和指數型基金）向來情有獨鍾。除了透過ETF投資全世界，他也透過寶來台灣卓越50基金（0050）布局台股部位，從金融海嘯以來，總報酬率已達到逾30%。
 
綠角於2007年創立「綠角財經筆記」部落格，2008年底即獲票選為最佳理財部落格第1名，深受網民歡迎。由於買股票需要花很多時間作功課，綠角認為，一般上班族無暇選股，不太可能跟法人在股市裡一較高下，因此，他崇尚指數化投資，因為買ETF不用擔心股市起伏，也不用憂慮基金成效。
 
「如果市場全年上漲50%，可能有一半投資人勝過平均，另一半必然會輸給平均。」對綠角來說，跟著指數賺錢的ETF雖然無法打敗大盤，取得超額報酬，但投資只需要每年考中上的成績，長年累積下來，回報同樣驚人。透過0050布局台股
熬過金融海嘯獲利逾30%著有《股海勝經》一書的綠角指出，ETF具備持股分散及成本低廉的特性，對一般

In [28]:
import pandas as pd
import requests, trafilatura
from newspaper import Article
from readability import Document
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import asyncio
import os
import json
from datetime import datetime

# Configuration
class Config:
    SOURCE_CSV = "聯發科爬到的新聞資料還有內文/google_news_results_playwright_merged_sorted.csv"
    OUTPUT_CSV = "聯發科爬到的新聞資料還有內文/聯發科_新聞_擷取結果.csv"
    CHECKPOINT_FILE = "crawler_checkpoint.json"
    ERROR_LOG = "crawler_errors.log"
    BATCH_SIZE = 10  # 每處理N筆存檔一次
class Checkpoint:
    def __init__(self):
        self.checkpoint_file = Config.CHECKPOINT_FILE
        self.load_checkpoint()
    
    def load_checkpoint(self):
        if os.path.exists(self.checkpoint_file):
            with open(self.checkpoint_file, 'r', encoding='utf-8') as f:
                self.data = json.load(f)
        else:
            self.data = {
                'last_index': -1,
                'timestamp': datetime.now().isoformat(),
                'failed_urls': [],
                'completed_urls': []
            }
    
    def save_checkpoint(self, index, url, success=True):
        self.data['last_index'] = index
        self.data['timestamp'] = datetime.now().isoformat()
        
        if success:
            if url not in self.data['completed_urls']:
                self.data['completed_urls'].append(url)
            if url in self.data['failed_urls']:
                self.data['failed_urls'].remove(url)
        else:
            if url not in self.data['failed_urls']:
                self.data['failed_urls'].append(url)
        
        with open(self.checkpoint_file, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)

def log_error(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(Config.ERROR_LOG, 'a', encoding='utf-8') as f:
        f.write(f"[{timestamp}] {message}\n")

async def fallback_with_browser(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto(url, timeout=30000)
            
            # 模擬滑鼠滾動
            for _ in range(10):
                await page.mouse.wheel(0, 1000)
                await page.wait_for_timeout(500)
            
            await page.wait_for_timeout(3000)
            text = await page.inner_text("body")
            return text
        except Exception as e:
            log_error(f"Playwright error on {url}: {str(e)}")
            raise e
        finally:
            await browser.close()

async def extract_from_url(index, row, checkpoint):
    name = row['title']
    url = row['link']
    print(f"\n🚀 正在處理第 {index + 1} 筆：{name}")
    print(f"🔗 URL: {url}")
    
    # 檢查是否已完成
    if url in checkpoint.data['completed_urls']:
        print("⏩ 此URL已處理過，跳過")
        return None
    
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        # 方法 1: trafilatura
        html = requests.get(url, headers=headers, timeout=15).text
        if html:
            text = trafilatura.extract(html)
            if text and len(text.strip()) > 100:
                print("✅ Trafilatura 擷取成功")
                checkpoint.save_checkpoint(index, url, success=True)
                return text
        
        # 方法 2: newspaper
        article = Article(url)
        article.download()
        article.parse()
        if article.text.strip():
            print("✅ Newspaper 擷取成功")
            checkpoint.save_checkpoint(index, url, success=True)
            return article.text
        
        # 方法 3: readability
        summary = BeautifulSoup(Document(html).summary(), "html.parser").get_text()
        if len(summary.strip()) > 100:
            print("✅ Readability 擷取成功")
            checkpoint.save_checkpoint(index, url, success=True)
            return summary
        
        # 方法 4: browser
        print("🚨 嘗試使用瀏覽器擷取...")
        text = await fallback_with_browser(url)
        if text:
            print("✅ Playwright 擷取成功")
            checkpoint.save_checkpoint(index, url, success=True)
            return text
            
    except Exception as e:
        error_msg = f"處理失敗 (第 {index + 1} 筆): {str(e)}"
        print(f"❌ {error_msg}")
        log_error(error_msg)
        checkpoint.save_checkpoint(index, url, success=False)
        return ""

async def main():
    # 初始化檢查點
    checkpoint = Checkpoint()
    print(f"📂 載入檢查點：上次處理到第 {checkpoint.data['last_index']} 筆")
    
    # 載入資料 - 增加錯誤處理
    try:
        if os.path.exists(Config.OUTPUT_CSV) and os.path.getsize(Config.OUTPUT_CSV) > 0:
            try:
                df = pd.read_csv(Config.OUTPUT_CSV)
                print("📊 載入既有的輸出檔案")
            except pd.errors.EmptyDataError:
                print("⚠️ 輸出檔案是空的，改為載入源檔案")
                df = pd.read_csv(Config.SOURCE_CSV)
                df['article_text'] = ""
        else:
            print("📊 建立新的輸出檔案")
            df = pd.read_csv(Config.SOURCE_CSV)
            df['article_text'] = ""
    except Exception as e:
        print(f"❌ 載入檔案時發生錯誤：{str(e)}")
        print("🔄 嘗試重新建立輸出檔案")
        df = pd.read_csv(Config.SOURCE_CSV)
        df['article_text'] = ""

    # 檢查是否成功載入資料
    if len(df) == 0:
        print("❌ 錯誤：沒有資料可處理")
        return
    
    print(f"📋 共載入 {len(df)} 筆資料")
    
    # 處理每一筆資料
    batch_count = 0
    for index, row in df.iterrows():
        # 檢查是否需要從斷點繼續
        if index <= checkpoint.data['last_index'] and str(row['article_text']).strip():
            print(f"⏩ 跳過第 {index + 1} 筆（已完成）")
            continue
            
        text = await extract_from_url(index, row, checkpoint)
        if text is not None:  # None 表示已處理過
            df.at[index, 'article_text'] = text
            batch_count += 1
            
            # 每處理N筆存檔一次
            if batch_count >= Config.BATCH_SIZE:
                try:
                    df.to_csv(Config.OUTPUT_CSV, index=False)
                    print(f"💾 已儲存最新 {Config.BATCH_SIZE} 筆結果")
                    batch_count = 0
                except Exception as e:
                    print(f"❌ 儲存檔案時發生錯誤：{str(e)}")
                    log_error(f"儲存檔案錯誤：{str(e)}")
    
    # 最後儲存
    try:
        df.to_csv(Config.OUTPUT_CSV, index=False)
        print(f"\n✅ 全部處理完成！")
        print(f"📊 結果儲存於：{Config.OUTPUT_CSV}")
        print(f"❌ 失敗紀錄：{Config.ERROR_LOG}")
    except Exception as e:
        print(f"❌ 最終儲存時發生錯誤：{str(e)}")
        log_error(f"最終儲存錯誤：{str(e)}")

# ✅ 執行
await main()

📂 載入檢查點：上次處理到第 -1 筆
📊 建立新的輸出檔案
📋 共載入 1304 筆資料

🚀 正在處理第 1 筆：2現象驗證 主升段來了！ - Smart自學網|財經好讀 - 出版品 - 密技 - 賺錢，跟著贏家走
🔗 URL: https://smart.businessweekly.com.tw/Books/special2.aspx?p=4&id=41869&type=1
✅ Trafilatura 擷取成功

🚀 正在處理第 2 筆：聯發科併雷凌 不影響市佔率
🔗 URL: https://ec.ltn.com.tw/article/paper/499240
✅ Trafilatura 擷取成功

🚀 正在處理第 3 筆：蔡宏圖表態：支持穩定的兩岸關係
🔗 URL: https://finance.ettoday.net/news/18779
✅ Trafilatura 擷取成功

🚀 正在處理第 4 筆：美林：記憶體族群 看不到春燕
🔗 URL: https://ec.ltn.com.tw/article/paper/560988
✅ Trafilatura 擷取成功

🚀 正在處理第 5 筆：張綱維稱不愛名女人　自爆「我單身」
🔗 URL: https://finance.ettoday.net/news/22202
✅ Trafilatura 擷取成功

🚀 正在處理第 6 筆：科技富豪避稅「星」趨勢　曹興誠、王泉仁入籍新加坡
🔗 URL: https://finance.ettoday.net/news/41310
✅ Trafilatura 擷取成功

🚀 正在處理第 7 筆：跌破眼鏡！聯發科併晨星- 財經要聞- 工商時報
🔗 URL: https://www.chinatimes.com/newspapers/20120623000003-260202
✅ Trafilatura 擷取成功

🚀 正在處理第 8 筆：聯發科併購晨星逾4成股權　震撼IC設計業
🔗 URL: https://finance.ettoday.net/news/64110
✅ Trafilatura 擷取成功

🚀 正在處理第 9 筆：聯發科砸1150億元 併購F-晨星
🔗 URL: https://www.moneyweekly.com.tw/Magazine/In

Future exception was never retrieved
future: <Future finished exception=Error('Connection closed')>
playwright._impl._api_types.Error: Connection closed
Future exception was never retrieved
future: <Future finished exception=Error('Connection closed')>
playwright._impl._api_types.Error: Connection closed


✅ Playwright 擷取成功

🚀 正在處理第 417 筆：中企拿補助搶技術 台科技堡壘告急
🔗 URL: https://ec.ltn.com.tw/article/paper/1370281
✅ Trafilatura 擷取成功

🚀 正在處理第 418 筆：聯發科4月營收雙衰退！月減9.98%、年減4.67%　仍站穩「200億元大關」
🔗 URL: https://finance.ettoday.net/news/1710037
✅ Trafilatura 擷取成功

🚀 正在處理第 419 筆：大立光、鴻海、聯發科...誰能取代台積電，成為台灣第二座護國神山？3條件檢視
🔗 URL: https://www.businesstoday.com.tw/article/category/80402/post/202006030031/
✅ Trafilatura 擷取成功

🚀 正在處理第 420 筆：上市公司員工薪酬揭露 華固員工去年平均領382萬稱霸 | Anue鉅亨 - 台股新聞
🔗 URL: https://news.cnyes.com/news/id/4484725
✅ Trafilatura 擷取成功
💾 已儲存最新 10 筆結果

🚀 正在處理第 421 筆：中階安卓手機5月性能榜出爐 聯發科處理器跑分奪冠
🔗 URL: https://www.chinatimes.com/realtimenews/20200603001823-260505
✅ Trafilatura 擷取成功

🚀 正在處理第 422 筆：外資看衰台積 「華為新禁令」輸家
🔗 URL: https://ec.ltn.com.tw/article/paper/1373721
✅ Trafilatura 擷取成功

🚀 正在處理第 423 筆：半導體廠薪資曝光！台積電員工平均年薪197.3萬元只排14強　晶焱296.8萬居冠
🔗 URL: https://finance.ettoday.net/news/1728046
✅ Trafilatura 擷取成功

🚀 正在處理第 424 筆：5奈米訂單穩啦- 財經要聞- 工商時報
🔗 URL: https://www.chinatimes.com/newspapers/20200511000170-260202
✅ Trafila

In [2]:
import pandas as pd

# 讀取 CSV 檔案
df = pd.read_csv("鴻海爬到的新聞資料還有內文/鴻海_新聞_擷取結果.csv")

# 顯示前五筆資料
print("📊 前五筆資料：")
print(df.head())

# 顯示每個欄位的資訊
print("\n📋 資料欄位資訊：")
print(df.info())
len(df)

📊 前五筆資料：
   date keyword                                title  \
0   NaN   鴻海 財經              知名部落客綠角 買ETF跟著指數賺 不用設停損   
1   NaN   鴻海 財經             EPS保8 鴻海獲利驚豔- 財經要聞- 工商時報   
2   NaN   鴻海 財經  基金女王不急著教女兒理財 林寶珠：一知半解的金融知識 反而容易揠苗助長   
3   NaN   鴻海 財經                       郭董諷巴菲特 有內線我也是神   
4   NaN   鴻海 財經                     「五股」小股東鬧場 郭董怒斥閃開   

                                                link publish_date  \
0  https://www.wealth.com.tw/articles/de60abf4-f4...   2011年1月11日   
1  https://www.chinatimes.com/newspapers/20110428...   2011年4月28日   
2  https://www.wealth.com.tw/articles/b1eca62c-95...   2011年4月30日   
3         https://ec.ltn.com.tw/article/paper/499221    2011年6月9日   
4         https://ec.ltn.com.tw/article/paper/499222    2011年6月9日   

                                        article_text  
0  財訊商城\n講座｜課程\n訂閱電子報\n會員中心\n登出\n財經\n政經\n金融圈\n科技\...  
1  如果您看到這個頁面，表示您的網路已遭到停止訪問本網站的權利。\n本站基於資訊安全以及保護網站...  
2  財訊商城\n講座｜課程\n訂閱電子報\n會員中心\n登出\n財經\n政經\n金融圈\n科技\...  
3  向仁寶陳瑞聰喊話\n鴻海搶了什麼

2948

In [26]:
# 讀取原始資料
df = pd.read_csv("鴻海爬到的新聞資料還有內文/鴻海_新聞_擷取結果.csv")

# 顯示原始資料筆數
print(f"原始資料筆數：{len(df)}")

# 過濾掉中時新聞網的資料
df_filtered = df[~df['link'].str.contains('chinatimes.com', na=False)]

# 顯示過濾後的資料筆數
print(f"過濾後資料筆數：{len(df_filtered)}")

# 儲存過濾後的資料
df_filtered.to_csv("鴻海爬到的新聞資料還有內文/鴻海_新聞_擷取結果_已過濾.csv", index=False)

# 顯示被移除的筆數
removed_count = len(df) - len(df_filtered)
print(f"已移除 {removed_count} 筆中時新聞網的資料")

原始資料筆數：2948
過濾後資料筆數：2817
已移除 131 筆中時新聞網的資料


In [27]:
# 過濾掉包含 "JavaScript is disabled" 的文章
df_filtered = df_filtered[~df_filtered['article_text'].str.contains('JavaScript is disabled', na=False)]

# 顯示過濾後的資料筆數
print(f"過濾後資料筆數：{len(df_filtered)}")

# 儲存過濾後的資料
df_filtered.to_csv("鴻海爬到的新聞資料還有內文/鴻海_新聞_擷取結果_已過濾.csv", index=False)

# 顯示被移除的筆數
removed_count = len(df) - len(df_filtered)
print(f"已移除 {removed_count} 筆包含 'JavaScript is disabled' 的文章")

過濾後資料筆數：2816
已移除 132 筆包含 'JavaScript is disabled' 的文章


In [32]:
# 讀取原始資料
df = pd.read_csv("台積電爬到的新聞資料還有內文/台積電_新聞_擷取結果.csv")

# 顯示原始資料筆數
print(f"原始資料筆數：{len(df)}")

# 過濾掉中時新聞網的資料
df_filtered = df[~df['link'].str.contains('chinatimes.com', na=False)]

# 顯示過濾後的資料筆數
print(f"過濾後資料筆數：{len(df_filtered)}")

# 儲存過濾後的資料
df_filtered.to_csv("台積電爬到的新聞資料還有內文/台積電_新聞_擷取結果_已過濾.csv", index=False)

# 顯示被移除的筆數
removed_count = len(df) - len(df_filtered)
print(f"已移除 {removed_count} 筆中時新聞網的資料")

原始資料筆數：3896
過濾後資料筆數：3176
已移除 720 筆中時新聞網的資料


In [33]:
# 檢查過濾後的資料中包含指定文字的筆數
js_disabled_count = df_filtered[df_filtered['article_text'].str.contains('JavaScript is disabled', na=False)].shape[0]

print(f"包含 'JavaScript is disabled' 的文章數量：{js_disabled_count}")

# 如果想查看這些文章的詳細資訊
js_disabled_articles = df_filtered[df_filtered['article_text'].str.contains('JavaScript is disabled', na=False)]
print("\n這些文章的資訊：")
print(js_disabled_articles[['title', 'link', 'date']])

包含 'JavaScript is disabled' 的文章數量：1

這些文章的資訊：
                          title                                    link  date
20  老闆租地種菜 員工度過金融危機 ｜ 公視新聞網 PNN  https://news.pts.org.tw/article/199841   NaN


In [34]:
# 過濾掉包含 "JavaScript is disabled" 的文章
df_filtered = df_filtered[~df_filtered['article_text'].str.contains('JavaScript is disabled', na=False)]

# 顯示過濾後的資料筆數
print(f"過濾後資料筆數：{len(df_filtered)}")

# 儲存過濾後的資料
df_filtered.to_csv("台積電爬到的新聞資料還有內文/台積電_新聞_擷取結果_已過濾.csv", index=False)

# 顯示被移除的筆數
removed_count = len(df) - len(df_filtered)
print(f"已移除 {removed_count} 筆包含 'JavaScript is disabled' 的文章")

過濾後資料筆數：3175
已移除 721 筆包含 'JavaScript is disabled' 的文章


聯發科

In [35]:
# 讀取原始資料
df = pd.read_csv("聯發科爬到的新聞資料還有內文/聯發科_新聞_擷取結果.csv")

# 顯示原始資料筆數
print(f"原始資料筆數：{len(df)}")

# 過濾掉中時新聞網的資料
df_filtered = df[~df['link'].str.contains('chinatimes.com', na=False)]

# 顯示過濾後的資料筆數
print(f"過濾後資料筆數：{len(df_filtered)}")

# 儲存過濾後的資料
df_filtered.to_csv("聯發科爬到的新聞資料還有內文/聯發科_新聞_擷取結果_已過濾.csv", index=False)

# 顯示被移除的筆數
removed_count = len(df) - len(df_filtered)
print(f"已移除 {removed_count} 筆中時新聞網的資料")

原始資料筆數：1304
過濾後資料筆數：983
已移除 321 筆中時新聞網的資料


In [36]:
# 過濾掉包含 "JavaScript is disabled" 的文章
df_filtered = df_filtered[~df_filtered['article_text'].str.contains('JavaScript is disabled', na=False)]

# 顯示過濾後的資料筆數
print(f"過濾後資料筆數：{len(df_filtered)}")

# 儲存過濾後的資料
df_filtered.to_csv("聯發科爬到的新聞資料還有內文/聯發科_新聞_擷取結果_已過濾.csv", index=False)

# 顯示被移除的筆數
removed_count = len(df) - len(df_filtered)
print(f"已移除 {removed_count} 筆包含 'JavaScript is disabled' 的文章")

過濾後資料筆數：983
已移除 321 筆包含 'JavaScript is disabled' 的文章
